In [1]:
import torch
from torchvision import transforms as T
from lightly.transforms import SimCLRTransform, DINOTransform, MAETransform, MoCoV2Transform, utils
from datasets import create_dataset
from models import SimCLRModel
import pytorch_lightning as pl
import os
import copy
import gc
import matplotlib.pyplot as plt
from itertools import product

e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Seed

In [2]:
SEED = 42

def seed_everything(seed: int=42):
    pl.seed_everything(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    ####### Normaly you would also need to seed those generators but `pytorch_lightning` does it in one func
    # random.seed(seed)
    # np.random.seed(seed)
    # torch.manual_seed(seed)
    ######
    torch.cuda.manual_seed(seed) # Don't know if pytorch lightning does this
    torch.cuda.manual_seed_all(seed) # Don't know if pytorch lightning does this
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(SEED)

Seed set to 42


# Transformacje

In [3]:
test_transform = T.v2.Compose(
    [
        T.Resize((224, 224)),
        T.v2.ToImage(),
        T.v2.ToDtype(torch.float32, scale=True),
        T.Normalize(
            mean=utils.IMAGENET_NORMALIZE["mean"],
            std=utils.IMAGENET_NORMALIZE["std"],
        )
    ]
)
scratch_transform = T.v2.Compose(
    [
        T.RandomResizedCrop((224, 224)),
        T.RandomHorizontalFlip(),
        T.v2.ToImage(),
        T.v2.ToDtype(torch.float32, scale=True),
        T.Normalize(
            mean=utils.IMAGENET_NORMALIZE["mean"],
            std=utils.IMAGENET_NORMALIZE["std"],
        )
    ]
)

simclr_transform = SimCLRTransform(
    input_size=(224, 224),
    vf_prob=0.5,
    rr_prob=0.5,
)

e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


# Zbiorki

In [4]:
train_full_cifar10_simclr, train_ssl_cifar10_simclr, train_cifar10_simclr, test_cifar10_simclr = create_dataset("CIFAR10", 0.9, simclr_transform, scratch_transform, test_transform, "data", False)

Length of entire train dataset:  50000
Length of SSL train dataset:  45000
Length of classification train dataset:  5000
Length of test dataset:  10000


In [9]:
train_full_cifar100_simclr, train_ssl_cifar100_simclr, train_cifar100_simclr, test_cifar100_simclr = create_dataset("CIFAR100", 0.9, simclr_transform, scratch_transform, test_transform, "data", False)

Length of entire train dataset:  50000
Length of SSL train dataset:  45000
Length of classification train dataset:  5000
Length of test dataset:  10000


# Hiperparametry

In [10]:
### PARAMETERS ###
BATCH_SIZE = 128
NUM_EPOCHS = 20
LEARNING_RATE = [0.001, 0.01]
BACKBONE_TYPE = ['pretrained', 'random']
NUM_WORKERS = 3
device = "gpu" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
#GARBAGE COLLECTOR FAJNA SPRAWA - BEZ NIEGO VRAMu BRAKUJE
if device == "gpu":
    torch.cuda.empty_cache()
    gc.collect()
    torch.set_float32_matmul_precision('high')

Using device: gpu


# CIFAR10

## Dataloadery i trening

In [6]:
dl_train_cifar10_simclr = torch.utils.data.DataLoader(
    train_ssl_cifar10_simclr,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    drop_last=True
)
dl_val_cifar10_simclr = torch.utils.data.DataLoader(
    test_cifar10_simclr,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

In [7]:
from pytorch_lightning.callbacks import Callback
import os

class SaveAtEpochsCallback(Callback):
    def __init__(self, save_epochs, dirpath="checkpoints"):
        super().__init__()
        self.save_epochs = set(save_epochs)
        self.dirpath = dirpath
        os.makedirs(self.dirpath, exist_ok=True)

    def on_train_epoch_end(self, trainer, pl_module):
        current_epoch = trainer.current_epoch  # epoka 0-based
        if current_epoch + 1 in self.save_epochs:
            filename = f"model_epoch_{current_epoch}.ckpt"
            path = os.path.join(self.dirpath, filename)
            trainer.save_checkpoint(path)
            print(f"Zapisano model po epoce {current_epoch}: {path}")

In [8]:
for lr, backbone_type in product(LEARNING_RATE, BACKBONE_TYPE):
    simclr_model = SimCLRModel(
        backbone_type=backbone_type + '_resnet18',
        lr=lr,
        max_epochs=NUM_EPOCHS,
    )

    dirpath = f"checkpoints/simclr_cifar10_{backbone_type}_{lr}"

    checkpoint_callback = SaveAtEpochsCallback(
        save_epochs=[10, 15, 20],
        dirpath=dirpath
    )

    trainer = pl.Trainer(
        accelerator='gpu',
        devices=1,
        max_epochs=NUM_EPOCHS,
        log_every_n_steps=1,
        callbacks=[checkpoint_callback],
        default_root_dir=dirpath,
    )

    trainer.fit(simclr_model, dl_train_cifar10_simclr, dl_val_cifar10_simclr)


e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\pytorch_ligh

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 19: 100%|██████████| 351/351 [03:34<00:00,  1.64it/s, v_num=1, train_loss=4.130, val_loss=3.730, val_ari=0.193, val_nmi=0.299, train_ari=0.178, train_nmi=0.275]

...

e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 3:  54%|█████▎    | 188/351 [11:22<09:51,  0.28it/s, v_num=0, train_loss=5.130, val_loss=4.160, val_ari=0.0626, val_nmi=0.107, train_ari=0.0306, train_nmi=0.0619]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

# CIFAR100

In [11]:
dl_train_cifar100_simclr = torch.utils.data.DataLoader(
    train_ssl_cifar100_simclr,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    drop_last=True
)
dl_val_cifar100_simclr = torch.utils.data.DataLoader(
    test_cifar100_simclr,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

In [12]:
from pytorch_lightning.callbacks import Callback
import os

class SaveAtEpochsCallback(Callback):
    def __init__(self, save_epochs, dirpath="checkpoints"):
        super().__init__()
        self.save_epochs = set(save_epochs)
        self.dirpath = dirpath
        os.makedirs(self.dirpath, exist_ok=True)

    def on_train_epoch_end(self, trainer, pl_module):
        current_epoch = trainer.current_epoch  # epoka 0-based
        if current_epoch + 1 in self.save_epochs:
            filename = f"model_epoch_{current_epoch}.ckpt"
            path = os.path.join(self.dirpath, filename)
            trainer.save_checkpoint(path)
            print(f"Zapisano model po epoce {current_epoch}: {path}")

In [13]:
for lr, backbone_type in product(LEARNING_RATE, BACKBONE_TYPE):
    torch.cuda.empty_cache()
    gc.collect()

    simclr_model = SimCLRModel(
        backbone_type=backbone_type + '_resnet18',
        lr=lr,
        max_epochs=NUM_EPOCHS,
    )

    dirpath = f"checkpoints/simclr_cifar100_{backbone_type}_{lr}"

    checkpoint_callback = SaveAtEpochsCallback(
        save_epochs=[10, 15, 20],
        dirpath=dirpath
    )

    trainer = pl.Trainer(
        accelerator='gpu',
        devices=1,
        max_epochs=NUM_EPOCHS,
        log_every_n_steps=1,
        callbacks=[checkpoint_callback],
        default_root_dir=dirpath,
    )

    trainer.fit(simclr_model, dl_train_cifar100_simclr, dl_val_cifar100_simclr)


e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 19: 100%|██████████| 351/351 [03:36<00:00,  1.62it/s, v_num=0, train_loss=4.140, val_loss=3.730, val_ari=0.0176, val_nmi=0.152, train_ari=0.0149, train_nmi=0.126]

...

e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


e:\programowanie\studia\sem6\WB2\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 19: 100%|██████████| 351/351 [03:36<00:00,  1.62it/s, v_num=0, train_loss=4.260, val_loss=3.790, val_ari=0.00948, val_nmi=0.0948, train_ari=0.00923, train_nmi=0.0806]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 351/351 [03:40<00:00,  1.59it/s, v_num=0, train_loss=4.260, val_loss=3.790, val_ari=0.00948, val_nmi=0.0948, train_ari=0.00923, train_nmi=0.0806]
